In [25]:
import pandas as pd
import geopandas as gpd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import itertools
import sys
sys.path.append('..')
from security_game.target import Target
from security_game.green_security_game import GreenSecurityGame
from security_game.infra_security_game import InfraSecurityGame

from solvers.mip import mip
from solvers.nash import nash
from solvers.double_oracle import double_oracle
from solvers.double_oracle_sf import double_oracle_sf
from solvers.no_regret import regret_matching
from solvers.simple_sse_lp import solve_sse_lp
from solvers.nfg_sse_lp import solve_general_sum_normal_form

from utils.random_utils import generate_random_utility_matrix_like, generate_random_target_utility_matrix_like

import time
from matplotlib import pyplot as plt
from collections import defaultdict, Counter
import copy

In [26]:
seeds = [1,2,3,4,5,6,7,8,9,10]

# GSG

In [3]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

## DO NFG Random Target Values (done)

In [9]:
schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)

supports=[]
defender_actions_sizes = []
attacker_actions_sizes = []
iterations_to_converge = []
iteration_times_li = []
gaps_li = []
num_defenders_li = []

for num_defenders in [1,2]:
    for seed in seeds:
        np.random.seed(seed)
        print(f"starting seed: {seed}")
        gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
        gsg.generate(num_attackers=1, 
                 num_defenders=num_defenders, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post) for i in range(num_defenders)],
                 num_timesteps=8, 
                 generate_utility_matrix=False,
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)

        D_a, D_d, u, A_a, A_d, c, iteration_times, gaps = double_oracle(gsg,eps=1e-12, verbose=False) #How to randomize for NFG DO?
        support = sum([1 for p in D_d if p!=0])
        supports.append(support)
        iterations_to_converge.append(c)
        defender_actions_sizes.append(len(A_d))
        attacker_actions_sizes.append(len(A_a))
        iteration_times_li.append(iteration_times)
        gaps_li.append(gaps)
        num_defenders_li.append(num_defenders)

starting seed: 1
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10
starting seed: 1
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10


In [13]:
df = pd.DataFrame()
seedli = seeds + seeds
df["seed"] = seedli
df["num_timesteps"] = [8 for i in range(len(seedli))]
df["num_defenders"] = num_defenders_li
df["num_attackers"] = [1 for i in range(len(seedli))]
df["num_clusters"] = [10 for i in range(len(seedli))]
df["dims"] = [7 for i in range(len(seedli))]
df["defense_time_threshold"] = [1 for i in range(len(seedli))]
df["force_return"] = [True for i in range(len(seedli))]
df["iterations_to_converge"] = iterations_to_converge
df["defender_actions_size"] = defender_actions_sizes
df["attacker_actions_size"] = attacker_actions_sizes
df["iteration_time"] = iteration_times_li
df["gap"] = gaps_li
df["def_support"] = supports


# Step 2: Explode list columns
exploded_df = df.explode(["iteration_time", "gap"], ignore_index=True)

# Optional: Add a column for iteration number within each grouped trial
exploded_df["iteration_number"] = exploded_df.groupby(
    ["seed","num_defenders"]
).cumcount()

# Reorder if desired
cols = [
    "seed", "num_timesteps","num_attackers", "num_defenders", "num_clusters", "dims", "force_return", "defense_time_threshold",
    "iteration_number", "iteration_time", "gap", 
    "iterations_to_converge", "def_support", "defender_actions_size", "attacker_actions_size"
]
exploded_df = exploded_df[cols]

# Make iteration number 1-indexed
exploded_df["iteration_number"] += 1
exploded_df[100:150]

,seed,num_timesteps,num_attackers,num_defenders,num_clusters,dims,force_return,defense_time_threshold,iteration_number,iteration_time,gap,iterations_to_converge,def_support,defender_actions_size,attacker_actions_size
100,2,8,1,2,10,7,True,1,10,0.096979,0.0,10,5,10,5
101,3,8,1,2,10,7,True,1,1,0.042066,0.641665,9,5,9,5
102,3,8,1,2,10,7,True,1,2,0.041631,0.641665,9,5,9,5
103,3,8,1,2,10,7,True,1,3,0.027831,0.367077,9,5,9,5
104,3,8,1,2,10,7,True,1,4,0.022873,0.395799,9,5,9,5
105,3,8,1,2,10,7,True,1,5,0.023052,0.293971,9,5,9,5
106,3,8,1,2,10,7,True,1,6,0.022732,0.233499,9,5,9,5
107,3,8,1,2,10,7,True,1,7,0.04009,0.108238,9,5,9,5
108,3,8,1,2,10,7,True,1,8,0.02604,0.209042,9,5,9,5
109,3,8,1,2,10,7,True,1,9,0.140026,0.0,9,5,9,5


In [14]:
exploded_df.to_csv("GSG_NFG_DO_RANDOM_TARGET_VALUES.csv")

## RM Algos w/ Matrix from random target values

### RM (done)

In [19]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
    gsg.generate(num_attackers=1, 
                 num_defenders=1, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(gsg.utility_matrix, runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=False, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [22]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [1 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,seed,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,interval_number,iteration_times,gaps
0,1,8,1,1,1,True,0,0.157047,0.217238
1,1,8,1,1,1,True,1,0.162086,0.147652
2,1,8,1,1,1,True,2,0.158237,0.131492
3,1,8,1,1,1,True,3,0.157952,0.1436
4,1,8,1,1,1,True,4,0.154361,0.128145
...,...,...,...,...,...,...,...,...,...
5698,10,8,1,1,1,True,526,0.195775,0.025191
5699,10,8,1,1,1,True,527,0.177078,0.024932
5700,10,8,1,1,1,True,528,0.199475,0.024586
5701,10,8,1,1,1,True,529,0.178296,0.02416


In [24]:
exploded_df.to_csv("GSG_NFG_RM_RANDOM_TARGET_VALUES_TO_MATRIX.csv")

### RM+

In [38]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
    gsg.generate(num_attackers=1, 
                 num_defenders=1, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(gsg.utility_matrix, runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=True, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.09701716361205281
Iteration 10 gap:  0.16620788881977794
Iteration 15 gap:  0.09217560795160157
Iteration 20 gap:  0.08981371418717876
Iteration 25 gap:  0.08770603573270819
Iteration 30 gap:  0.09304471229975453
Iteration 35 gap:  0.0760975624451577
Iteration 40 gap:  0.09746563433856015
Iteration 45 gap:  0.09419015984422019
Iteration 50 gap:  0.08163126357064521
Iteration 55 gap:  0.09158927092615077
Iteration 60 gap:  0.08705076800382089
Iteration 65 gap:  0.07379470911438624
Iteration 70 gap:  0.07542773281669396
Iteration 75 gap:  0.08003904381851878


KeyboardInterrupt: 

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [1 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_NFG_RMP_RANDOM_TARGET_VALUES_TO_MATRIX.csv")

### PRM+

In [39]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
    gsg.generate(num_attackers=1, 
                 num_defenders=1, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(gsg.utility_matrix, runtime=120, interval=5, iterations=10000, averaging=2, alternations=True, plus=True, predictive=True, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.28093741988591686
Iteration 10 gap:  0.10000197038903874
Iteration 15 gap:  0.11728873809408075
Iteration 20 gap:  0.1252580499793361
Iteration 25 gap:  0.12686085210197362
Iteration 30 gap:  0.11394375814375324
Iteration 35 gap:  0.10758817952768007
Iteration 40 gap:  0.10868328540355354


KeyboardInterrupt: 

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [1 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_NFG_PRMP_RANDOM_TARGET_VALUES_TO_MATRIX.csv")

### RM random matrix

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=1, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=False,
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    random_matrix = np.random.uniform(low=np.min(gsg.utility_matrix), high=np.max(gsg.utility_matrix), size=gsg.utility_matrix.shape)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(random_matrix, runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=False, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [1 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_NFG_RM_RANDOM_MATRIX.csv")

### RM+ random matrix

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=1, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=False,
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    random_matrix = np.random.uniform(low=np.min(gsg.utility_matrix), high=np.max(gsg.utility_matrix), size=gsg.utility_matrix.shape)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(random_matrix, runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=True, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [1 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_NFG_RMP_RANDOM_MATRIX.csv")

### PRM+ random matrix

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=1, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=False,
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    random_matrix = np.random.uniform(low=np.min(gsg.utility_matrix), high=np.max(gsg.utility_matrix), size=gsg.utility_matrix.shape)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(random_matrix, runtime=120, interval=5, iterations=10000, averaging=2, alternations=True, plus=True, predictive=True, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [1 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_NFG_PRMP_RANDOM_MATRIX.csv")

### GSG SF DO Random target values (done)

In [10]:
schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_animal_value":  2350, 
    "defender_animal_value": 22966, 
    "defender_step_cost": 0, 
}

df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)

supports=[]
iterations_to_converge = []
defender_actions_sizes = []
attacker_actions_sizes = []
iteration_times_li = []
gaps_li = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7, escape_line_points=None)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=False, 
             defense_time_threshold=1, 
             force_return=True, 
             **schedule_form_kwargs,
             **general_sum_kwargs)
for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    sdict_copy = copy.deepcopy(gsg.schedule_form_dict)
    sdict_copy["target_utilities"] = generate_random_target_utility_matrix_like(gsg.schedule_form_dict["target_utilities"], general_sum=False, respect_sign_roles=True)
    # print(gsg.schedule_form_dict)
    # print(sdict_copy)
    # print("starting double oracle sf")
    # print(sdict_copy["target_utilities"])
    D_a, D_d, u, A_a, A_d, c, iteration_times, gaps = double_oracle_sf(sdict_copy,eps=1e-12, verbose=False)
    # print("Done")
    support = sum([1 for p in D_d if p!=0])
    supports.append(support)
    iterations_to_converge.append(c)
    defender_actions_sizes.append(len(A_d))
    attacker_actions_sizes.append(len(A_a))
    iteration_times_li.append(iteration_times)
    gaps_li.append(gaps)

starting seed: 1
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10


In [11]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "num_clusters": [10 for i in range(len(seeds))],
    "dims": [7 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "def_support": supports,
    "iterations_to_converge": iterations_to_converge,
    "iteration_time": iteration_times_li,
    "gap": gaps_li,
    "defender_actions_size": defender_actions_sizes,
    "attacker_actions_size": attacker_actions_sizes
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_time", "gap"], ignore_index=True)

# Optional: Add a column for iteration number within each grouped trial
exploded_df["iteration_number"] = exploded_df.groupby(
    ["seed"]
).cumcount()

# Reorder if desired
cols = ["seed",
    "num_timesteps", "num_defenders", "num_clusters", "dims", "defense_time_threshold",
    "iteration_number", "iteration_time", "gap", 
    "iterations_to_converge", "def_support", "defender_actions_size", "attacker_actions_size"
]
exploded_df = exploded_df[cols]

# Make iteration number 1-indexed
exploded_df["iteration_number"] += 1

exploded_df

,seed,num_timesteps,num_defenders,num_clusters,dims,defense_time_threshold,iteration_number,iteration_time,gap,iterations_to_converge,def_support,defender_actions_size,attacker_actions_size
0,1,8,2,10,7,1,1,0.002074,0.320218,4,1,2,3
1,1,8,2,10,7,1,2,0.002062,0.399867,4,1,2,3
2,1,8,2,10,7,1,3,0.002521,0.186256,4,1,2,3
3,1,8,2,10,7,1,4,0.002317,0.0,4,1,2,3
4,2,8,2,10,7,1,1,0.002141,0.290276,12,5,7,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,10,8,2,10,7,1,6,0.004575,0.04406,10,5,9,6
78,10,8,2,10,7,1,7,0.005159,0.052337,10,5,9,6
79,10,8,2,10,7,1,8,0.006253,0.434904,10,5,9,6
80,10,8,2,10,7,1,9,0.005241,0.052492,10,5,9,6


In [18]:
exploded_df.to_csv("GSG_SF_DO_RANDOM_TARGET_VALUES.csv")

### RM

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
    gsg.generate(num_attackers=1, 
                 num_defenders=2, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(gsg.schedule_form_dict["defender_utility_matrix"], runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=False, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_SF_RANDOM_TARGET_VALUES_TO_MATRIX.csv")

### RM+

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
    gsg.generate(num_attackers=1, 
                 num_defenders=2, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(gsg.schedule_form_dict["defender_utility_matrix"], runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=True, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_SF_RMP_RANDOM_TARGET_VALUES_TO_MATRIX.csv")

### PRM+

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
    gsg.generate(num_attackers=1, 
                 num_defenders=2, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(gsg.schedule_form_dict["defender_utility_matrix"], runtime=120, interval=5, iterations=10000, averaging=2, alternations=True, plus=True, predictive=True, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_SF_PRMP_RANDOM_TARGET_VALUES_TO_MATRIX.csv")

### RM random matrix

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=True,
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    random_matrix = np.random.uniform(low=np.min(gsg.schedule_form_dict["defender_utility_matrix"]), high=np.max(gsg.schedule_form_dict["defender_utility_matrix"]), size=gsg.schedule_form_dict["defender_utility_matrix"].shape)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(random_matrix, runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=False, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_SF_RM_RANDOM_MATRIX.csv")

### RM+ random matrix

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=True,
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    random_matrix = np.random.uniform(low=np.min(gsg.schedule_form_dict["defender_utility_matrix"]), high=np.max(gsg.schedule_form_dict["defender_utility_matrix"]), size=gsg.schedule_form_dict["defender_utility_matrix"].shape)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(random_matrix, runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=True, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_SF_RMP_RANDOM_MATRIX.csv")

### PRM+ random matrix

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=True,
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    random_matrix = np.random.uniform(low=np.min(gsg.schedule_form_dict["defender_utility_matrix"]), high=np.max(gsg.schedule_form_dict["defender_utility_matrix"]), size=gsg.schedule_form_dict["defender_utility_matrix"].shape)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(random_matrix, runtime=120, interval=5, iterations=10000, averaging=2, alternations=True, plus=True, predictive=True, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_SF_PRMP_RANDOM_MATRIX.csv")

# ISG

In [27]:
gdf = gpd.read_file("chinatown_infra.geojson")

# Step 1: Handle relevant columns
infra_columns = [
    "id", "name", "power", "man_made", "amenity",
    "generator:method", "generator:source", "geometry"
]
available_columns = [col for col in infra_columns if col in gdf.columns]
gdf = gdf[available_columns].copy()

# Step 2: Extract generator type if present
gdf["generator_type"] = gdf.get("generator:method")
if "generator_type" not in gdf.columns or gdf["generator_type"].isnull().all():
    gdf["generator_type"] = gdf.get("generator:source")

# Step 3: Construct unified 'type' column
gdf["type"] = gdf.get("power")
if "amenity" in gdf.columns:
    gdf["type"] = gdf["type"].combine_first(gdf["amenity"])
if "man_made" in gdf.columns:
    gdf["type"] = gdf["type"].combine_first(gdf["man_made"])

# Step 4: Refine generator classification (solar vs. other)
gdf.loc[(gdf["type"] == "generator") & (gdf["generator_type"] == "photovoltaic"), "type"] = "solar_generator"
gdf.loc[(gdf["type"] == "generator") & (gdf["generator_type"] == "solar"), "type"] = "solar_generator"

# Step 5: Drop raw columns now that 'type' is finalized
df_simple = gdf[["id", "name", "type", "geometry"]].copy()

# Step 6: Separate nodes and ways
df_nodes = df_simple[df_simple["id"].str.contains("node")].copy()
df_nodes["x"] = df_nodes.geometry.x
df_nodes["y"] = df_nodes.geometry.y
df_nodes = df_nodes.drop(columns=["geometry"])

df_ways = df_simple[df_simple["id"].str.contains("way")].copy()
df_ways = df_ways.set_geometry("geometry").to_crs("EPSG:32618")
df_ways["centroid"] = df_ways.geometry.centroid
df_ways = df_ways.set_geometry("centroid").to_crs("EPSG:4326")
df_ways["x"] = df_ways.geometry.x
df_ways["y"] = df_ways.geometry.y
df_ways = df_ways.drop(columns=["geometry", "centroid"])

# Step 7: Combine nodes and ways
df_combined = pd.concat([df_nodes, df_ways], ignore_index=True)
df_combined = pd.concat([df_nodes, df_ways], ignore_index=True)
ny_blocks_gdf =  gpd.read_file("tl_2020_36_tabblock20.shp")
INFRA_WEIGHTS = {
    # Power Infrastructure
    "plant": 1.5,
    "generator": 1.35,
    "solar_generator": 0.95,
    "substation": 1.45,
    "transformer": 1.25,
    "tower": 1.1,
    "pole": 0.85,
    "line": 1.0,
    "minor_line": 0.9,
    "cable": 0.95,
    "switchgear": 1.2,
    "busbar": 0.8,
    "bay": 0.85,
    "converter": 1.05,
    "insulator": 0.75,
    "portal": 0.75,
    "connection": 0.7,
    "compensator": 1.0,
    "rectifier": 0.95,
    "inverter": 0.95,
    "storage": 0.9,

    # Healthcare
    "hospital": 1.5,
    "clinic": 1.35,

    # Education
    "school": 1.25,
    "university": 1.4,

    # Water & Sanitation
    "water_works": 1.45,
    "wastewater_plant": 1.4,

    # Government & Emergency Services
    "fire_station": 1.3,
    "police": 1.4,
    "courthouse": 1.2,

    # Critical Infrastructure
    "bunker_silo": 1.0,

    # Communications
    "communications_tower": 1.25,
}


## DO NFG Random Target Values (done)

In [9]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 3,
    "defender_penalty_factor": 3,
    "extra_coverage_weight":0.9
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

# Bounding box for Hoboken, NJ
# bbox_hoboken_small = (40.752635, 40.745600, -74.030386,-74.043903)
bbox_hoboken_low = (40.745411, 40.735486, -74.025857,-74.041479)
bbox_hoboken_east = (40.748337, 40.734641,-74.022961,-74.031286)
bbox_downtown = (40.718721, 40.714078, -73.996074, -74.002651)
bbox_downtown_large = (40.7215, 40.710, -73.9935, -74.010)
# 40.7060, -74.0140, 40.7205, -73.9935
college_police = (40.743293077312465, -74.02670221027175)
police_station = (40.73768931976651, -74.02990519431108)
traffic_police = (40.7366602084371, -74.03449866349136)
downtown_station = (40.71232433042349, -74.00187755238431)
fifth_ave_station = (40.71637413934789, -73.9973285259067)
fifth_precinct = (40.71625547686622, -73.99736909131171)
booking_station = (40.716191530904815, -74.00102237385177)
police_plaza = (40.71236124409745, -74.00173715463521)
troop_nyc = (40.71657885026091, -74.00641139014367)
first_precinct = (40.720411300417446, -74.0070247584372)

num_timesteps_li = []
num_attackers_li = []
num_defenders_li = []
supports=[]
dims_li = []
dts = []
frs = []
defender_actions_sizes = []
attacker_actions_sizes = []
iterations_to_converge = []
iteration_times_li = []
gaps_li = []
i=0

for num_defenders in [1,2,3]:
    for seed in seeds:
        isg = InfraSecurityGame(df_combined, ny_blocks_gdf, INFRA_WEIGHTS, bbox=bbox_downtown_large)
        isg.generate(num_attackers=1,
                     num_defenders=num_defenders,
                     home_base_assignments=[(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza) for i in range(num_defenders)],
                     num_timesteps=num_timesteps,
                     generate_utility_matrix=False,
                     generate_actions=False,
                     force_return=True,
                     defense_time_threshold=dt,
                     **general_sum_kwargs,
                     **schedule_form_kwargs)
        D_a, D_d, u, A_a, A_d, c, iteration_times, gaps = double_oracle(isg,eps=1e-12, verbose=False)
        support = sum([1 for p in D_d if p!=0])
        supports.append(support)
        num_timesteps_li.append(num_timesteps)
        num_attackers_li.append(num_attackers)
        num_defenders_li.append(num_defenders)
        dts.append(dt)
        frs.append(fr)
        iterations_to_converge.append(c)
        defender_actions_sizes.append(len(A_d))
        attacker_actions_sizes.append(len(A_a))
        iteration_times_li.append(iteration_times)
        gaps_li.append(gaps)
        print(f"{i}/179: timesteps:{num_timesteps}, num_attackers:{num_attackers}, num_defenders:{num_defenders}, dt:{dt}, fr:{fr}, support:{support}")
        i+=1

starting seed: 1
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10
starting seed: 1
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10


In [13]:
df = pd.DataFrame()
seedli = seeds + seeds
df["seed"] = seedli
df["num_timesteps"] = [8 for i in range(len(seedli))]
df["num_defenders"] = num_defenders_li
df["num_attackers"] = [1 for i in range(len(seedli))]
df["num_clusters"] = [10 for i in range(len(seedli))]
df["dims"] = [7 for i in range(len(seedli))]
df["defense_time_threshold"] = [1 for i in range(len(seedli))]
df["force_return"] = [True for i in range(len(seedli))]
df["iterations_to_converge"] = iterations_to_converge
df["defender_actions_size"] = defender_actions_sizes
df["attacker_actions_size"] = attacker_actions_sizes
df["iteration_time"] = iteration_times_li
df["gap"] = gaps_li
df["def_support"] = supports


# Step 2: Explode list columns
exploded_df = df.explode(["iteration_time", "gap"], ignore_index=True)

# Optional: Add a column for iteration number within each grouped trial
exploded_df["iteration_number"] = exploded_df.groupby(
    ["seed","num_defenders"]
).cumcount()

# Reorder if desired
cols = [
    "seed", "num_timesteps","num_attackers", "num_defenders", "num_clusters", "dims", "force_return", "defense_time_threshold",
    "iteration_number", "iteration_time", "gap", 
    "iterations_to_converge", "def_support", "defender_actions_size", "attacker_actions_size"
]
exploded_df = exploded_df[cols]

# Make iteration number 1-indexed
exploded_df["iteration_number"] += 1
exploded_df[100:150]

,seed,num_timesteps,num_attackers,num_defenders,num_clusters,dims,force_return,defense_time_threshold,iteration_number,iteration_time,gap,iterations_to_converge,def_support,defender_actions_size,attacker_actions_size
100,2,8,1,2,10,7,True,1,10,0.096979,0.0,10,5,10,5
101,3,8,1,2,10,7,True,1,1,0.042066,0.641665,9,5,9,5
102,3,8,1,2,10,7,True,1,2,0.041631,0.641665,9,5,9,5
103,3,8,1,2,10,7,True,1,3,0.027831,0.367077,9,5,9,5
104,3,8,1,2,10,7,True,1,4,0.022873,0.395799,9,5,9,5
105,3,8,1,2,10,7,True,1,5,0.023052,0.293971,9,5,9,5
106,3,8,1,2,10,7,True,1,6,0.022732,0.233499,9,5,9,5
107,3,8,1,2,10,7,True,1,7,0.04009,0.108238,9,5,9,5
108,3,8,1,2,10,7,True,1,8,0.02604,0.209042,9,5,9,5
109,3,8,1,2,10,7,True,1,9,0.140026,0.0,9,5,9,5


In [14]:
exploded_df.to_csv("GSG_NFG_DO_RANDOM_TARGET_VALUES.csv")

## RM Algos w/ Matrix from random target values

### RM (done)

In [19]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
    gsg.generate(num_attackers=1, 
                 num_defenders=1, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(gsg.utility_matrix, runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=False, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [22]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [1 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,seed,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,interval_number,iteration_times,gaps
0,1,8,1,1,1,True,0,0.157047,0.217238
1,1,8,1,1,1,True,1,0.162086,0.147652
2,1,8,1,1,1,True,2,0.158237,0.131492
3,1,8,1,1,1,True,3,0.157952,0.1436
4,1,8,1,1,1,True,4,0.154361,0.128145
...,...,...,...,...,...,...,...,...,...
5698,10,8,1,1,1,True,526,0.195775,0.025191
5699,10,8,1,1,1,True,527,0.177078,0.024932
5700,10,8,1,1,1,True,528,0.199475,0.024586
5701,10,8,1,1,1,True,529,0.178296,0.02416


In [24]:
exploded_df.to_csv("GSG_NFG_RM_RANDOM_TARGET_VALUES_TO_MATRIX.csv")

### RM+

In [38]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
    gsg.generate(num_attackers=1, 
                 num_defenders=1, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(gsg.utility_matrix, runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=True, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.09701716361205281
Iteration 10 gap:  0.16620788881977794
Iteration 15 gap:  0.09217560795160157
Iteration 20 gap:  0.08981371418717876
Iteration 25 gap:  0.08770603573270819
Iteration 30 gap:  0.09304471229975453
Iteration 35 gap:  0.0760975624451577
Iteration 40 gap:  0.09746563433856015
Iteration 45 gap:  0.09419015984422019
Iteration 50 gap:  0.08163126357064521
Iteration 55 gap:  0.09158927092615077
Iteration 60 gap:  0.08705076800382089
Iteration 65 gap:  0.07379470911438624
Iteration 70 gap:  0.07542773281669396
Iteration 75 gap:  0.08003904381851878


KeyboardInterrupt: 

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [1 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_NFG_RMP_RANDOM_TARGET_VALUES_TO_MATRIX.csv")

### PRM+

In [39]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
    gsg.generate(num_attackers=1, 
                 num_defenders=1, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(gsg.utility_matrix, runtime=120, interval=5, iterations=10000, averaging=2, alternations=True, plus=True, predictive=True, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.28093741988591686
Iteration 10 gap:  0.10000197038903874
Iteration 15 gap:  0.11728873809408075
Iteration 20 gap:  0.1252580499793361
Iteration 25 gap:  0.12686085210197362
Iteration 30 gap:  0.11394375814375324
Iteration 35 gap:  0.10758817952768007
Iteration 40 gap:  0.10868328540355354


KeyboardInterrupt: 

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [1 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_NFG_PRMP_RANDOM_TARGET_VALUES_TO_MATRIX.csv")

### RM random matrix

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=1, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=False,
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    random_matrix = np.random.uniform(low=np.min(gsg.utility_matrix), high=np.max(gsg.utility_matrix), size=gsg.utility_matrix.shape)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(random_matrix, runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=False, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [1 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_NFG_RM_RANDOM_MATRIX.csv")

### RM+ random matrix

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=1, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=False,
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    random_matrix = np.random.uniform(low=np.min(gsg.utility_matrix), high=np.max(gsg.utility_matrix), size=gsg.utility_matrix.shape)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(random_matrix, runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=True, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [1 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_NFG_RMP_RANDOM_MATRIX.csv")

### PRM+ random matrix

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=1, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=False,
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    random_matrix = np.random.uniform(low=np.min(gsg.utility_matrix), high=np.max(gsg.utility_matrix), size=gsg.utility_matrix.shape)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(random_matrix, runtime=120, interval=5, iterations=10000, averaging=2, alternations=True, plus=True, predictive=True, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [1 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_NFG_PRMP_RANDOM_MATRIX.csv")

### GSG SF DO Random target values (done)

In [10]:
schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_animal_value":  2350, 
    "defender_animal_value": 22966, 
    "defender_step_cost": 0, 
}

df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)

supports=[]
iterations_to_converge = []
defender_actions_sizes = []
attacker_actions_sizes = []
iteration_times_li = []
gaps_li = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7, escape_line_points=None)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=False, 
             defense_time_threshold=1, 
             force_return=True, 
             **schedule_form_kwargs,
             **general_sum_kwargs)
for seed in seeds:
    print(f"starting seed: {seed}")
    np.random.seed(seed)
    sdict_copy = copy.deepcopy(gsg.schedule_form_dict)
    sdict_copy["target_utilities"] = generate_random_target_utility_matrix_like(gsg.schedule_form_dict["target_utilities"], general_sum=False, respect_sign_roles=True)
    # print(gsg.schedule_form_dict)
    # print(sdict_copy)
    # print("starting double oracle sf")
    # print(sdict_copy["target_utilities"])
    D_a, D_d, u, A_a, A_d, c, iteration_times, gaps = double_oracle_sf(sdict_copy,eps=1e-12, verbose=False)
    # print("Done")
    support = sum([1 for p in D_d if p!=0])
    supports.append(support)
    iterations_to_converge.append(c)
    defender_actions_sizes.append(len(A_d))
    attacker_actions_sizes.append(len(A_a))
    iteration_times_li.append(iteration_times)
    gaps_li.append(gaps)

starting seed: 1
starting seed: 2
starting seed: 3
starting seed: 4
starting seed: 5
starting seed: 6
starting seed: 7
starting seed: 8
starting seed: 9
starting seed: 10


In [11]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "num_clusters": [10 for i in range(len(seeds))],
    "dims": [7 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "def_support": supports,
    "iterations_to_converge": iterations_to_converge,
    "iteration_time": iteration_times_li,
    "gap": gaps_li,
    "defender_actions_size": defender_actions_sizes,
    "attacker_actions_size": attacker_actions_sizes
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_time", "gap"], ignore_index=True)

# Optional: Add a column for iteration number within each grouped trial
exploded_df["iteration_number"] = exploded_df.groupby(
    ["seed"]
).cumcount()

# Reorder if desired
cols = ["seed",
    "num_timesteps", "num_defenders", "num_clusters", "dims", "defense_time_threshold",
    "iteration_number", "iteration_time", "gap", 
    "iterations_to_converge", "def_support", "defender_actions_size", "attacker_actions_size"
]
exploded_df = exploded_df[cols]

# Make iteration number 1-indexed
exploded_df["iteration_number"] += 1

exploded_df

,seed,num_timesteps,num_defenders,num_clusters,dims,defense_time_threshold,iteration_number,iteration_time,gap,iterations_to_converge,def_support,defender_actions_size,attacker_actions_size
0,1,8,2,10,7,1,1,0.002074,0.320218,4,1,2,3
1,1,8,2,10,7,1,2,0.002062,0.399867,4,1,2,3
2,1,8,2,10,7,1,3,0.002521,0.186256,4,1,2,3
3,1,8,2,10,7,1,4,0.002317,0.0,4,1,2,3
4,2,8,2,10,7,1,1,0.002141,0.290276,12,5,7,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,10,8,2,10,7,1,6,0.004575,0.04406,10,5,9,6
78,10,8,2,10,7,1,7,0.005159,0.052337,10,5,9,6
79,10,8,2,10,7,1,8,0.006253,0.434904,10,5,9,6
80,10,8,2,10,7,1,9,0.005241,0.052492,10,5,9,6


In [18]:
exploded_df.to_csv("GSG_SF_DO_RANDOM_TARGET_VALUES.csv")

### RM

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
    gsg.generate(num_attackers=1, 
                 num_defenders=2, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(gsg.schedule_form_dict["defender_utility_matrix"], runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=False, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_SF_RANDOM_TARGET_VALUES_TO_MATRIX.csv")

### RM+

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
    gsg.generate(num_attackers=1, 
                 num_defenders=2, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(gsg.schedule_form_dict["defender_utility_matrix"], runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=True, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_SF_RMP_RANDOM_TARGET_VALUES_TO_MATRIX.csv")

### PRM+

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []
for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
    gsg.generate(num_attackers=1, 
                 num_defenders=2, 
                 home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
                 num_timesteps=8, 
                 generate_utility_matrix=True, 
                 random_target_values=True,
                 defense_time_threshold=1, 
                 generate_actions=False, 
                 force_return=True, 
                 general_sum=False, 
                 **schedule_form_kwargs)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(gsg.schedule_form_dict["defender_utility_matrix"], runtime=120, interval=5, iterations=10000, averaging=2, alternations=True, plus=True, predictive=True, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_SF_PRMP_RANDOM_TARGET_VALUES_TO_MATRIX.csv")

### RM random matrix

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=True,
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    random_matrix = np.random.uniform(low=np.min(gsg.schedule_form_dict["defender_utility_matrix"]), high=np.max(gsg.schedule_form_dict["defender_utility_matrix"]), size=gsg.schedule_form_dict["defender_utility_matrix"].shape)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(random_matrix, runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=False, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_SF_RM_RANDOM_MATRIX.csv")

### RM+ random matrix

In [ ]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

# Lobeke National Park Bounding Box
# lat_min, lon_min = 2.05522, 15.8790
# lat_max, lon_max = 2.2837, 16.2038

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]


boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)
sangha_river = [(2.2837, 16.16283352464626),(2.053, 16.066212728001727)]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

gaps_li = []
interval_times_li = []

gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             random_target_values=True,
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

for seed in seeds:
    print(f"starting seed {seed}")
    np.random.seed(seed)
    random_matrix = np.random.uniform(low=np.min(gsg.schedule_form_dict["defender_utility_matrix"]), high=np.max(gsg.schedule_form_dict["defender_utility_matrix"]), size=gsg.schedule_form_dict["defender_utility_matrix"].shape)
    print("game done generating, running")
    D_d, U, gaps, interval_times = regret_matching(random_matrix, runtime=120, interval=5, iterations=10000, averaging=0, alternations=False, plus=True, predictive=False, verbose=True)
    gaps_li.append(gaps)
    interval_times_li.append(interval_times)

starting seed 1
game done generating, running
Iteration 5 gap:  0.21723831433169066
Iteration 10 gap:  0.14765174788643387
Iteration 15 gap:  0.1314924131369276
Iteration 20 gap:  0.1435998288772009
Iteration 25 gap:  0.12814458742604062
Iteration 30 gap:  0.1718830760932452
Iteration 35 gap:  0.16831852444284062
Iteration 40 gap:  0.16956568288001417
Iteration 45 gap:  0.13604157333659822
Iteration 50 gap:  0.09908805556236855
Iteration 55 gap:  0.07813052646761287
Iteration 60 gap:  0.08332852474281444
Iteration 65 gap:  0.07646435928419515
Iteration 70 gap:  0.05947192522284883
Iteration 75 gap:  0.05231808026048684
Iteration 80 gap:  0.047424855799586885
Iteration 85 gap:  0.05648477198658325
Iteration 90 gap:  0.058452964853945744
Iteration 95 gap:  0.06096410254131007
Iteration 100 gap:  0.07723093188378699
Iteration 105 gap:  0.09702496515689274
Iteration 110 gap:  0.11276424676903507
Iteration 115 gap:  0.12304712278316787
Iteration 120 gap:  0.11589526099401376
Iteration 125 g

In [36]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps": [8 for i in range(len(seeds))],
    "num_attackers": [1 for i in range(len(seeds))],
    "num_defenders": [2 for i in range(len(seeds))],
    "defense_time_threshold": [1 for i in range(len(seeds))],
    "force_return": [True for i in range(len(seeds))],
    "iteration_times": interval_times_li,
    "gaps": gaps_li
})

# Step 2: Explode list columns
exploded_df = df.explode(["iteration_times", "gaps"], ignore_index=True)

# Optional: Add a column for iteration number within each trial
exploded_df["interval_number"] = exploded_df.groupby(["seed"]).cumcount()

# Reorder if desired
cols = ["seed","num_timesteps", "num_attackers", "num_defenders", "defense_time_threshold", "force_return", "interval_number", "iteration_times", "gaps"]
exploded_df = exploded_df[cols]

exploded_df["interval_number"] = exploded_df["interval_number"]+1
exploded_df["iteration_number"] = exploded_df["interval_number"]*5
exploded_df

,num_timesteps,num_attackers,num_defenders,defense_time_threshold,force_return,iteration_number,iteration_times,gaps
0,8,1,1,1,True,0,0.192131,0.217238
1,8,1,1,1,True,1,0.21313,0.147652
2,8,1,1,1,True,2,0.171854,0.131492
3,8,1,1,1,True,3,0.180722,0.1436
4,8,1,1,1,True,4,0.178561,0.128145
5,8,1,1,1,True,5,0.188964,0.171883
6,8,1,1,1,True,6,0.238595,0.168319
7,8,1,1,1,True,7,0.222445,0.169566
8,8,1,1,1,True,0,0.211734,0.32868
9,8,1,1,1,True,1,0.170227,0.191657


In [ ]:
exploded_df.to_csv("GSG_SF_RMP_RANDOM_MATRIX.csv")

### PRM+ random matrix